#### keep in mind that this proof-of-concept for leveraging a pre-trained model the data collecting by me pre-prosecced and labled with their respective dialcet with this format dialect:sentence

#### Bringing in deps

In [23]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [2]:
# Load the Arabert model and tokenizer
model_name = "aubmindlab/bert-base-arabertv02"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

C:\Users\ahmed\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahmed\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
# Load dataset
dataset_path = "ad_dialect_data(2).txt" 
with open(dataset_path, "r", encoding="utf-8") as file:
    dataset_lines = file.readlines()

In [8]:
max_length = 0

with open(dataset_path, "r", encoding="utf-8") as file:
    for line in file:
        # Check if the line contains ":"
        if ":" in line:
            # formatted as dialect:sentence
            sentence = line.strip().split(":")[1]
            length = len(sentence.split())
            max_length = max(max_length, length)

print("Maximum Sequence Length:", max_length)

Maximum Sequence Length: 139


In [24]:
# Tokenize the dataset
tokenized_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=dataset_path,
    block_size=139,  # max_length
)

# Create a data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,  # We're not doing masked language modeling (change if needed)
)


C:\Users\ahmed\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [28]:
# Fine-tuning training arguments
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=263, training_loss=0.040581046855041734, metrics={'train_runtime': 945.6368, 'train_samples_per_second': 2.225, 'train_steps_per_second': 0.278, 'total_flos': 106285781508096.0, 'train_loss': 0.040581046855041734, 'epoch': 1.0})

In [29]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_model(1)")
tokenizer.save_pretrained("fine_tuned_model(1)")


('fine_tuned_model(1)\\tokenizer_config.json',
 'fine_tuned_model(1)\\special_tokens_map.json',
 'fine_tuned_model(1)\\vocab.txt',
 'fine_tuned_model(1)\\added_tokens.json',
 'fine_tuned_model(1)\\tokenizer.json')